In [ ]:
# First install the necessary library by running this command
# Either here or in your terminal (without the exclamation mark if using terminal)
%pip install datasets transformers torch torchvision --extra-index-url https://download.pytorch.org/whl/cu121


In [ ]:

# Now let's load a dataset - CIFAR-10 is a classic so let's go with that!
from datasets import load_dataset

dataset = load_dataset('cifar10')

In [ ]:
# We can take a quick look at one example from our dataset to see what it includes
example = next(iter(dataset['train']))
print(example.keys())


In [ ]:
from torchvision import transforms
import numpy as np
from PIL import Image

# Transformations commonly used when training image classifiers.
train_transforms = transforms.Compose([
    transforms.ToTensor(), # Convert images to PyTorch tensors
    transforms.Normalize((0.5,), (0.5,))  # Normalize values to be between -1 and 1 for faster convergence.
])

# Let's try stacking those image tensors into one big multi-dimensional torch.Tensor!
def transform_image(example_batch):
    # Transform each image individually and stack them into a single tensor.
    example_batch['img'] = torch.stack([train_transforms(img) for img in example_batch['img']])
    return example_batch

# Apply the transformation function again with batching.
train_dataset = dataset['train'].map(transform_image, batched=True)


In [ ]:
# Transforming labels into tensor and making sure they are of type int not string.
def transform_labels(example_batch):
    example_batch['label'] = torch.tensor([int(label) for label in example_batch['label']])
    return example_batch

# Apply this updated function to convert our label strings to int tensors.
train_dataset = train_dataset.map(transform_labels, batched=True)

# Let's create that DataLoader again.
train_loader = DataLoader(train_dataset.shuffle(load_from_cache_file=False), batch_size=64)

In [ ]:
from torch.utils.data import DataLoader

# This little incantation tells our dataset how we want each example returned - as PyTorch tensors!
train_dataset.set_format(type='torch', columns=['img', 'label'])

# Now we can create a DataLoader directly from the Hugging Face dataset.
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# And now let's try grabbing some images and labels again:
for batch in train_loader:
    images, labels = batch['img'], batch['label']
    break  # We escape after snatching just one batch!

# Attempting visualization once more...
show_images(images, labels.numpy())

In [ ]:
import torch.nn as nn

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        # Our first convolution block will have 3 input channels (for R,G,B colors)
        # And we'll start with 8 filters (a choice made on whim and experience!)
        self.conv1 = nn.Conv2d(3, 8, kernel_size=5)

        # We'll use good ol' max pooling after convolutions
        self.pool = nn.MaxPool2d(kernel_size=2)

        # Second convolution block will take those 8 incoming channels
        # And transform them into 16 channels.
        self.conv2 = nn.Conv2d(8, 16, kernel_size=5)

        # Fully-connected or "dense" layers at the end to produce our actual class scores.
        # The CIFAR-10 dataset comes with 10 classes (like airplanes, dogs, cats) so that's the output size we need.
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # We'll run the input tensor through each operation in sequence
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))

        # Before we can use fully connected layers,
        # we have to flatten our tensor into a vector
        x = x.view(-1, 16 * 5 * 5)

        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))

        # The final layer will return raw scores for each class
        x = self.fc3(x)

        return x

In [ ]:
model = Net()

In [ ]:
import torch.optim as optim

# We'll use Cross-Entropy Loss for multi-class classification
criterion = nn.CrossEntropyLoss()

# Adam is a popular choice for an optimizer - feel free to experiment with others!
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
import torch.nn.functional as F
num_epochs = 10  # feel free to tweak this as you see fit

# Training loop:
for epoch in range(num_epochs):
    running_loss = 0.0

    # Loop over each batch
    for i, batch in enumerate(train_loader):
        images, labels = batch['img'], batch['label']

        # Zero parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)

        # Calculate loss
        loss = criterion(outputs, labels)

        # Backward pass
        loss.backward()

        # Optimize
        optimizer.step()

        # Print stats
        running_loss += loss.item()

    print(f"Epoch {epoch + 1} loss: {running_loss/len(train_loader)}")

print('Finished Training!')

In [ ]:
# Make sure to apply the same transformations to keep things consistent.
valid_dataset = dataset['test'].map(transform_image, batched=True)
valid_dataset = valid_dataset.map(transform_labels, batched=True)
# Set format so PyTorch can understand this Hugging Face dataset
valid_dataset.set_format(type='torch', columns=['img', 'label'])

# Valid_loader will fetch batches from valid_dataset
valid_loader = DataLoader(valid_dataset.shuffle(load_from_cache_file=False), batch_size=64)

In [ ]:
correct_predictions = 0 
total_predictions = 0
model.eval() # Switch model to evaluation mode

with torch.no_grad(): # We're not backpropagating through validation, so don't compute gradients. correct_predictions = 0 `total_predictions = 0
    for batch in valid_loader: 
        images, labels = batch['img'], batch['label'] 
        outputs = model(images)
        # The class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)

        total_predictions += labels.size(0)  # Update total count
        correct_predictions += (predicted == labels).sum().item() 

In [ ]:
import matplotlib.pyplot as plt
import torchvision
def imshow(img):
    img = img / 2 + 0.5  # Unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()
# Let's grab a single batch from our validation dataset.
for batch in valid_loader:
    images, labels = batch['img'], batch['label'] 
    # Run the images through the model to get outputs
    outputs = model(images)

    imshow(torchvision.utils.make_grid(images))
    # The class with the highest energy is what we choose as prediction
    _, preds = torch.max(outputs.data, 1)

    preds_classes = [valid_dataset.features['label'].names[pred] for pred in preds]

    print('Predicted: ', ' '.join(preds_classes))
    break

In [ ]:
import torch.nn as nn
import torch.optim as optim
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.fc1 = nn.Linear(784, 32)
        self.fc2 = nn.Linear(32, 10)
    def forward(self, x):
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = torch.relu(x)
        x = self.fc2(x)
        x = torch.log_softmax(x, dim=1)
        return x

In [ ]:
import torch
from torch.utils.data import DataLoader
import torchvision
# Download and load the MNIST dataset
train_dataset = torchvision.datasets.MNIST(
    root='./data', train=True, transform=torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize((0.1307,), (0.3081,))]))
# Define the data loaders for train and test sets
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
# Define the model and move it to the GPU (if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
# Define an optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.NLLLoss()
# Training loop
epochs = 20
for epoch in range(epochs):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    train_loss = running_loss / len(train_loader)
    print('Epoch [{}] Loss: {:.4f}'.format(epoch+1, train_loss))

In [ ]:
class CNN(nn.Module):
    def __init__(self, input_shape, num_classes):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(input_shape[0], 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(4096, 128)  # Update these dimensions
        self.fc2 = nn.Linear(128, num_classes)
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.dropout1(x)
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)  # Use log_softmax for CrossEntropyLoss
        return output


In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from tqdm import tqdm
dataset_name = 'cifar10'
num_classes = 10
batch_size = 32
learning_rate = 0.001
num_epochs = 10
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# Load the dataset
from datasets import load_dataset
dataset = load_dataset(dataset_name)['train']
# Get unique labels (assuming they are in a 'labels' column)
unique_labels = set(dataset['label'])

# Create a mapping from labels to integers
label_to_tensor = {label: torch.tensor(idx) for idx, label in enumerate(unique_labels)}

# Function to convert a label to its tensor representation
def label_to_tensor_map(label):
    return label_to_tensor[label]

# Convert the labels in the dataset
dataset = dataset.map(lambda example: {'label': label_to_tensor_map(example['label'])})
# Apply data augmentation and normalization
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
def preprocess_train(example_batch):
    """Apply train_transforms across a batch."""
    example_batch["img"] = [
        transform(image.convert("RGB")) for image in example_batch["img"]
    ]
    return example_batch

# Load the data in batches
dataset = dataset.map(preprocess_train, batched=True)

dataset = dataset.with_format('torch', columns=['img', 'label'])
train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
# Create the CNN model
model = CNN((3, 32, 32), num_classes).to(device)
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# Train the model
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    for data in tqdm(train_dataloader):
        # Ensure data and target are in the correct format
        inputs, labels = data['img'].to(device), data['label'].to(device)

        optimizer.zero_grad()
        output = model(inputs)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    
    print(f"Epoch {epoch + 1} | Loss: {epoch_loss / len(train_dataloader)}")